In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

60480

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2520)

In [15]:
x[:,:2]

matrix([[ 2359.40210634,  2337.01256984],
        [ 2289.06489292,  2294.88891528],
        [ 2883.33833614,  2872.965419  ],
        [ 2856.7205003 ,  2897.36290514],
        [ 2989.5127047 ,  2886.67967098],
        [ 2755.20249667,  2757.71269425],
        [ 2434.70287806,  2373.94636866],
        [ 2404.74286605,  2441.8102877 ],
        [ 3153.06425139,  3202.20093408],
        [ 3071.24632571,  3101.49609424],
        [ 2796.84764738,  2664.79023625],
        [ 2431.02788311,  2505.99992656],
        [ 2284.14488821,  2187.46896874],
        [ 2534.90654419,  2464.4075785 ],
        [ 2283.42356399,  2353.64255004],
        [ 1843.51004281,  1798.20815298],
        [ 2408.43820312,  2414.03983482],
        [ 2367.70036212,  2419.71275118],
        [ 2679.31127806,  2671.64071345],
        [ 2630.67573585,  2610.10425748],
        [ 2825.33776339,  2742.55550243],
        [ 2641.53488651,  2646.86945136],
        [ 2393.1215448 ,  2248.67933905],
        [ 2380.13108071,  2425.991

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e+01, 7e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 26
 AA' NZ     : 3.250e+02
 Factor NZ  : 3.510e+02
 Factor Ops : 6.201e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.01323211e+08  0.00000000e+00  9.65e+03 1.78e+03  1.00e+06     0s
   1  -5.35834080e+07 -3.37411973e+07  1.47e+03 2.70e+02  2.77e+05     0s
   2  -3.93235665e+07 -4.05849589e+07  2.60e+01 4.79e+00  3.34e+04     0s
   3  -3.93014237e+07 -3.93641141e+07  2.67e-02 4.93e-03  1.1

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.0064874262837873e-10,
 3088.605671087576,
 1.574674157370647e-10,
 0.0016459825032209639,
 2731.9396099095347,
 0.002731274178865952,
 4.751795880083137e-08,
 1.3719183356606342e-05,
 1.88344043904792e-11,
 2526.6224172383695,
 1.883440441612141e-11,
 1.8834404034046738e-11,
 1.883440426064563e-11,
 1.8942409279337887e-11,
 3695.3849242621227,
 1777.8791918767909,
 0.027951930410322456,
 3049.2174794568796,
 0.0011137087964882726,
 0.002731210675695102,
 0.0027289037809411915,
 1460.0104588756594,
 1161.0253831707626,
 0.013710629801638702,
 0.27736800430920994,
 0.171174976837827,
 2121.382337351787,
 2353.59648155441,
 0.006175830673448637,
 0.00602203833642815,
 370.06811575902447,
 2413.191848121531,
 2276.8717556605984,
 50.18847557788135,
 29.798738051839052,
 8.51644771433527e-07,
 8.516426529600472e-07,
 8.516384078568531e-07,
 0.03248312076955398,
 1841.7083165388592,
 1824.2945266815088,
 935.8736555953129,
 0.7250116055913054,
 5.72217480513558,
 2826.7001411335664,
 0.02